## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [0]:
import os
import os.path
import glob
import numpy as np
import scipy as sp
import scipy.io
import scipy.signal
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pickle

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data

    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def PreprocessTestData(data_fl, ref_fl):
    """
    Preprocess algorithm data test.
    Args:
        data_fl: (str) filepath to a troika .mat file (DATA).
        ref_fls: (str) filepath to a troika .mat file (REF).
    Returns:
        Numpy arrays for targets, features, sig, subs
        
    """    
    fs=125
    win_len = 8
    win_shift = 2
    
    
    
    sig = LoadTroikaDataFile(data_fl)
    ref = scipy.io.loadmat(ref_fl)["BPM0"]
    ref = np.array([x[0] for x in ref])
    subject_name = os.path.basename(data_fl).split('.')[0]        
    start_indxs, end_indxs = get_indxs(sig.shape[1], len(ref), fs, win_len,win_shift)
    targets, features, sigs, subs = [], [], [], []
    for i, s in enumerate(start_indxs):
        start_i =  start_indxs[i]
        end_i = end_indxs[i]

        ppg = sig[0, start_i:end_i]            
        accx = sig[1, start_i:end_i]
        accy = sig[2, start_i:end_i]
        accz = sig[3, start_i:end_i]

        ppg = BandpassFilter(ppg)
        accx = BandpassFilter(accx)
        accy = BandpassFilter(accy)
        accz = BandpassFilter(accz)

        feature, ppg, accx, accy, accz = Featurize(ppg, accx, accy, accz)

        sigs.append([ppg, accx, accy, accz])
        targets.append(ref[i])
        features.append(feature)
        subs.append(subject_name)
        
    return (np.array(targets), np.array(features), sigs, subs)

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Featurize(ppg, accx, accy, accz):
    """A partial featurization of the accelerometer signal.
    
    Args:
        accx: (np.array) x-channel of the accelerometer.
        accy: (np.array) y-channel of the accelerometer.
        accz: (np.array) z-channel of the accelerometer.
        fs: (number) the sampling rate of the accelerometer
        
    Returns:
        numpy array of accelerometer features
    """

    fs = 125
    n = len(ppg) * 4
    minutes = 60.0

    freqs = np.fft.rfftfreq(n, 1/fs)
    fft = np.abs(np.fft.rfft(ppg,n))
    fft[freqs <= 40/minutes] = 0.0
    fft[freqs >= 240/minutes] = 0.0
    
    acc_mag = np.sqrt(accx**2 + accy**2 + accz**2)
    acc_fft = np.abs(np.fft.rfft(acc_mag, n))
    acc_fft[freqs <= 40/minutes] = 0.0
    acc_fft[freqs >= 240/minutes] = 0.0
    
    ppg_feature = freqs[np.argmax(fft)]
    acc_feature = freqs[np.argmax(acc_fft)]
    
    return (np.array([ppg_feature, acc_feature]), ppg, accx, accy, accz)

def Evaluate():
    """
    Evaluate alorithm performance.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    global reg
    reg = TrainRegressor()
    
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def BandpassFilter(signal, fs = 125):   
    """Bandpass filter the signal. """
    minutes = 60.0

    b, a = scipy.signal.butter(3, (40/minutes, 240/minutes), btype='bandpass', fs=fs)
    return scipy.signal.filtfilt(b, a, signal)

def get_indxs(sig_len, ref_len, fs=125, win_len_s=10, win_shift_s=2):
    """
    Find start and end index to iterate over a set of signals
    """
    if ref_len < sig_len:
        n = ref_len
    else:
        n = sig_len
    
    start_indxs = (np.cumsum(np.ones(n) * fs * win_shift_s) - fs * win_shift_s).astype(int)
    end_indxs = start_indxs + win_len_s * fs
    return (start_indxs, end_indxs)

def TrainRegressor():
    """
    Applies regression using RandomForestRegressor
    Parameters: estimators=400, depth=20
    KFold plits = 5
    
    Return:
        Regression
    """
    fs=125
    win_len = 8
    win_shift = 2
    
    data_fls, ref_fls = LoadTroikaDataset()
    targets, features, sigs, subs = [], [], [], []
    for data_fl, ref_fl in (zip(data_fls, ref_fls)):
        
        sig = LoadTroikaDataFile(data_fl)
        ref = scipy.io.loadmat(ref_fl)["BPM0"]
        ref = np.array([x[0] for x in ref])
        subject_name = os.path.basename(data_fl).split('.')[0]        
        start_indxs, end_indxs = get_indxs(sig.shape[1], len(ref), fs, win_len,win_shift)
        for i, s in enumerate(start_indxs):
            start_i =  start_indxs[i]
            end_i = end_indxs[i]

            ppg = sig[0, start_i:end_i]            
            accx = sig[1, start_i:end_i]
            accy = sig[2, start_i:end_i]
            accz = sig[3, start_i:end_i]

            ppg = BandpassFilter(ppg)
            accx = BandpassFilter(accx)
            accy = BandpassFilter(accy)
            accz = BandpassFilter(accz)

            feature, ppg, accx, accy, accz = Featurize(ppg, accx, accy, accz)

            sigs.append([ppg, accx, accy, accz])
            targets.append(ref[i])
            features.append(feature)
            subs.append(subject_name)
            
    targets = np.array(targets)
    features = np.array(features)
    
    regression = RandomForestRegressor(n_estimators=400,max_depth=20)
    lf = KFold(n_splits=5)
    splits = lf.split(features,targets,subs)

    for i, (train_idx, test_idx) in enumerate(splits):
        X_train, y_train = features[train_idx], targets[train_idx]
        X_test, y_test = features[test_idx], targets[test_idx]
        regression.fit(X_train, y_train)
    
    return regression

def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    Run algorithm.
    
    Args:
        data_fl: (str) filepath to a troika .mat file (DATA).
        ref_fls: (str) filepath to a troika .mat file (REF).

    Return:
        Numpy arrary with error and confidence
    """
    fs = 125
    win_len = 8
    win_shift = 2    
       
    targets, features, sigs, subs = PreprocessTestData(data_fl, ref_fl)

    error, confidence = [], []
    for i,feature in enumerate(features):
        est = reg.predict(np.reshape(feature, (1, -1)))[0]
        ppg, accx, accy, accz = sigs[i]        
        
        n = len(ppg) * 4
        minutes = 60.0
        freqs = np.fft.rfftfreq(n, 1/fs)
        fft = np.abs(np.fft.rfft(ppg,n))
        fft[freqs <= 40/minutes] = 0.0
        fft[freqs >= 240/minutes] = 0.0
    
        est_fs = est / 55.0
        fs_win = 30  / 60.0
        fs_win_e = (freqs >= est_fs - fs_win) & (freqs <= est_fs +fs_win)
        conf = np.sum(fft[fs_win_e])/np.sum(fft)
        
        error.append(np.abs((est-targets[i])))
        confidence.append(conf)
    return np.array(error), np.array(confidence)
Evaluate()

9.507958762630615

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.



**CODE DESCRIPTION**

- We defined and used functions to Load and pre-process data, add metrics, featurize, evaluate, filter application (bandpass), get index, train regresor and to run the algorithm.

**DATA DESCRIPTION**

We are using troika dataset to build our algorithm: datasets/troika/training_data

- The main purpose of the dataset is PPG-based heart rate estimation. For this task, typically three sensor modalities are used: a) the PPG-sensor itself, b) 3D-accelerometer embedded in the same device as the PPG-sensor, used to compensate motion artefacts, and c) ECG which provides heart rate ground truth. It is common practice in related work to use a sliding window approach (window length: 8 seconds, window shift: 2 seconds) [6-10]. This means that all data signal is segmented with this sliding window, and the goal is to determine the heart rate on each 8-second window segment.
- We use the mean for both PPG Channels in order to obtain more accurate results.
- ECG Signal with one channel
- 3-axis accelerometer, 3 channels for each axis x,y,z respectively.

**ALGORITHM DESCRIPTION**

**How the algorithm works?**
    
- Used RandomForestRegressor from sklearn to featurize data
- Estimates pulse rate from the PPG signal and a 3-axis accelerometer.
- Assumes pulse rate will be restricted between 40BPM (beats per minute) and 240BPM
- Produces an estimation confidence. A higher confidence value means that this estimate should be more accurate than an estimate with a lower confidence value.
- Produces an output at least every 2 seconds.

**Specific aspects of the physiology that it takes advantage of:**

Change in light measures: PPG signals can be used for measuring heart rate, when the blood passes, light is emitted by the PPG sensor. When capillaries in the wrist fill with blood and the ventricles contract, the photodetector will detect when the reflected light stops and this oscillating waveform is the pulse rate.

**Algorithm Outputs & Cavets:**

- Estimated frequency (BPM) 
- The optimaly results in respect of the confidence score prediction, **Higher confidence means a better estimate**. The best 90% of the estimates are above the 10th percentile confidence.


**Caveats:**

Confidence rate is calculated only based on magnitud of a small area that contains the estimated spectral frequency relative to the sum magnitude of the entire spectrum.

**Common Failures**

Failure when PPG takes signals other than heart beat rate, possible due to movements. In order to manage this,  the algoritm uses accelerations measurement as well. 


**Algorithm Performance**


- As we used **RandomForestRegressor** we used the following **parameters**: estimators=400, depth=20 KFold plits = 5. We also preprocessed data test featurizing and applying bandpass filter.
- Performance was given by calculating the mean absolute error between the heart rate estimation and the the heart rate reference from the ECG sensors at 90% availability. 
- Algorithm Final Score was **9.51** with the parameters mentioned above. The most promesing approach  to evaluating the algorithm would be a machine learning approach.

-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 